**data from:** 

***https://visaguide.world/visa-free-countries/salvadoran-passport/***

***https://www.worlddata.info/cost-of-living.php***




In [3]:
import requests
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup

In [16]:
r = requests.get('https://www.worlddata.info/cost-of-living.php')
soup = BeautifulSoup(r.text, 'html.parser')

world_data = [tag for tag in soup.find_all(['tr'])]
country_bucket = []
column_names = ['Countries'] # scraped country names from https://www.worlddata.info/cost-of-living.php

n = 1
while n < len(world_data):
    countries = world_data[n].a.text
    n+=1
    country_bucket.append(countries)

country_bucket = sorted(country_bucket)

df = pd.DataFrame(country_bucket, columns=column_names)
df

,Countries
0,Albania
1,Algeria
2,Australia
3,Austria
4,Azerbaijan
...,...
95,United Kingdom
96,United States
97,Uzbekistan
98,Vanuatu


In [ ]:
r = requests.get('https://visaguide.world/visa-free-countries/salvadoran-passport/')
soup = BeautifulSoup(r.text, 'html.parser')

print(soup.find('ul'))